# Homework Session 11 Neural networks

### part 1

The purpose of the below is to classify days over years 2017-2018 by their corresponding mobility patterns between 10 zones in Taipei (quantified by an aggregated temporal network of subway ridership flows across the city)

In [ ]:
import numpy as np
import pandas as pd
import keras
from keras.datasets import mnist
from keras.models import Model, Sequential
from keras.layers import Input, Dense
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from sklearn.preprocessing import MinMaxScaler
from keras.utils import np_utils
from keras import backend as K
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

import numpy as np
from nltk.tokenize import RegexpTokenizer
from keras.models import Sequential, load_model
from keras.layers import LSTM
from keras.layers.core import Dense, Activation
from keras.optimizers import RMSprop
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import pickle
import heapq

In [ ]:
#read the data
TNet=pd.read_csv('taipeiD_TNet2.csv',header=None);

In [ ]:
TNet.head() 
#each row represents a 10x10 adjacency matrix of the normalized Taipei subway mobility network between 10 zones flattened into a 100x1 row corresponding to a single day
#days start at jan-1-2017

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,0.017943,0.005415,0.003590,0.008316,0.007859,0.012942,0.012196,0.019543,0.001196,0.003327,...,0.002529,0.001533,0.001860,0.002375,0.005408,0.008922,0.003945,0.011075,0.005073,0.012708
1,0.021283,0.005215,0.003530,0.009359,0.007803,0.014288,0.011185,0.019044,0.001383,0.003499,...,0.002803,0.001757,0.001783,0.002549,0.005515,0.009650,0.003596,0.009618,0.005946,0.013709
2,0.028988,0.006511,0.005591,0.012970,0.007816,0.015878,0.010973,0.015768,0.002252,0.005388,...,0.004649,0.002555,0.002672,0.004291,0.007385,0.009558,0.004293,0.008791,0.010040,0.016301
3,0.029534,0.006471,0.005615,0.013017,0.007717,0.016098,0.011182,0.015815,0.002325,0.005443,...,0.004611,0.002473,0.002635,0.004195,0.007255,0.009487,0.004316,0.008729,0.010296,0.016437
4,0.029333,0.006525,0.005727,0.013098,0.007692,0.016358,0.011000,0.015677,0.002344,0.005527,...,0.004694,0.002515,0.002677,0.004222,0.007269,0.009921,0.004387,0.008923,0.010381,0.016914


In [ ]:
#convert to an array and scale the data
X=np.array(TNet);

In [ ]:
X=MinMaxScaler(feature_range=(0, 1), copy=True).fit_transform(X)

In [ ]:
X.shape

(669, 100)

In [ ]:
#define day of the week corresponding to each day of observation; 0-Sunday, 1-Monday,...,6-Saturday
y=np.array(range(669))%7; y[:10]

array([0, 1, 2, 3, 4, 5, 6, 0, 1, 2])

In [ ]:
yc=np_utils.to_categorical(y) #get categorical binary variables isSunday, isMonday,...
yc[:5]

array([[1., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0.]], dtype=float32)

In [ ]:
X_test=X[400:,:]; X_train=X[:400,:]; #split the data into training and test
y_test=yc[400:,:]; y_train=yc[:400,:]

In [ ]:
X_test

array([[0.90159786, 0.72082394, 0.68945875, ..., 0.31791065, 0.7516474 ,
        0.6480811 ],
       [0.80536476, 0.72076218, 0.67706797, ..., 0.40164665, 0.74135895,
        0.64908372],
       [0.81765599, 0.66149903, 0.67597867, ..., 0.35410987, 0.72873928,
        0.63215062],
       ...,
       [0.90658665, 0.81779439, 0.57798706, ..., 0.25836145, 0.87024431,
        0.72583969],
       [0.88836671, 0.81415027, 0.6014524 , ..., 0.24613308, 0.88847445,
        0.79435192],
       [0.8981997 , 0.7926253 , 0.5901963 , ..., 0.27293832, 0.86042211,
        0.74722888]])

## Task 1. Classify weekdays/weekends
Label the rows with ones for weekends, zeros for weekdays.
Train a neural network with 4 layers of 30,10,3 and 1 (output) neurons over the training sample against this label, evaluating its performance over the test sample. Report the acheived accuracy (categorical) over the test sample

First three layers use relu activation function, last one - sigmoid.
Use loss='binary_crossentropy', optimizer='adam', 100 epochs, batch_size=20. 

In [ ]:
y_label = np.array([(i % 7 > 4) for i in range(X.shape[0])]).astype(int)
y_train_label = y_label[:400]
y_test_label = y_label[400:]

In [ ]:
dim = X_train.shape[1]
np.random.seed(2023)
model = Sequential()
model.add(Dense(30, activation='relu', input_dim=dim))
model.add(Dense(10, activation='relu', input_dim=dim))
model.add(Dense(3, activation='relu', input_dim=dim))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train_label, validation_data=(X_test, y_test_label), epochs=100, batch_size=20, verbose=2)
predict_test = model.predict(X_test)

Epoch 1/100
20/20 - 1s - loss: 0.5648 - accuracy: 0.7150 - val_loss: 0.5264 - val_accuracy: 0.7175 - 1s/epoch - 53ms/step
Epoch 2/100
20/20 - 0s - loss: 0.5119 - accuracy: 0.7150 - val_loss: 0.4907 - val_accuracy: 0.7175 - 52ms/epoch - 3ms/step
Epoch 3/100
20/20 - 0s - loss: 0.4798 - accuracy: 0.7150 - val_loss: 0.4605 - val_accuracy: 0.7175 - 54ms/epoch - 3ms/step
Epoch 4/100
20/20 - 0s - loss: 0.4483 - accuracy: 0.7150 - val_loss: 0.4329 - val_accuracy: 0.7212 - 59ms/epoch - 3ms/step
Epoch 5/100
20/20 - 0s - loss: 0.4238 - accuracy: 0.7375 - val_loss: 0.4036 - val_accuracy: 0.7323 - 54ms/epoch - 3ms/step
Epoch 6/100
20/20 - 0s - loss: 0.3939 - accuracy: 0.7675 - val_loss: 0.3808 - val_accuracy: 0.7472 - 71ms/epoch - 4ms/step
Epoch 7/100
20/20 - 0s - loss: 0.3689 - accuracy: 0.8000 - val_loss: 0.3655 - val_accuracy: 0.7398 - 54ms/epoch - 3ms/step
Epoch 8/100
20/20 - 0s - loss: 0.3487 - accuracy: 0.8375 - val_loss: 0.3605 - val_accuracy: 0.7398 - 57ms/epoch - 3ms/step
Epoch 9/100
20/20

In [ ]:
#Report accuracy 
test=pd.DataFrame()
test['predict_test']=y_test_label
test['predict_label'] = predict_test.flatten()
test['predict_label_']=1.0*(test['predict_label']>0.5)
 #Test accuracy
1.0*sum((test['predict_test']==1)==(test['predict_label']>0.5))/len(test['predict_label'])

0.9628252788104089

## Task 2. Classify all days of the week
Train a neural network against the origial categorical label. Use 5 layers of 40,15,5 and 7 (outputs, representing probabilities for a current input to correspond to each of the weekdays) neurons over the training sample, evaluating its performance over the test sample (use 'categorical_accurary'). Report the acheived accuracy (categorical) over the test sample.

First three layers use relu activation function, last one - sigmoid.
Use loss='binary_crossentropy', optimizer='adam', 200 epochs, batch_size=20

In [ ]:
dim = X_train.shape[1]
np.random.seed(2023)
model = Sequential()
model.add(Dense(40, activation='relu', input_dim=dim))
model.add(Dense(15, activation='relu', input_dim=dim))
model.add(Dense(5, activation='relu', input_dim=dim))
model.add(Dense(7, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=200, batch_size=20, verbose=2)
prob = model.predict(X_test)

Epoch 1/200
20/20 - 1s - loss: 0.6904 - categorical_accuracy: 0.1550 - val_loss: 0.6677 - val_categorical_accuracy: 0.1450 - 804ms/epoch - 40ms/step
Epoch 2/200
20/20 - 0s - loss: 0.6552 - categorical_accuracy: 0.1425 - val_loss: 0.6362 - val_categorical_accuracy: 0.1450 - 72ms/epoch - 4ms/step
Epoch 3/200
20/20 - 0s - loss: 0.6189 - categorical_accuracy: 0.1425 - val_loss: 0.5946 - val_categorical_accuracy: 0.1487 - 110ms/epoch - 5ms/step
Epoch 4/200
20/20 - 0s - loss: 0.5727 - categorical_accuracy: 0.1425 - val_loss: 0.5440 - val_categorical_accuracy: 0.1487 - 57ms/epoch - 3ms/step
Epoch 5/200
20/20 - 0s - loss: 0.5132 - categorical_accuracy: 0.1975 - val_loss: 0.4770 - val_categorical_accuracy: 0.1970 - 78ms/epoch - 4ms/step
Epoch 6/200
20/20 - 0s - loss: 0.4557 - categorical_accuracy: 0.2575 - val_loss: 0.4311 - val_categorical_accuracy: 0.2788 - 64ms/epoch - 3ms/step
Epoch 7/200
20/20 - 0s - loss: 0.4212 - categorical_accuracy: 0.2750 - val_loss: 0.4077 - val_categorical_accuracy:

In [ ]:
#Report the acheived accuracy (categorical) over the test sample.
print('Accuracy Rate: %f' % accuracy_score(y_test.argmax(axis = 1),prob.argmax(axis = 1)))
print('')
print('Confusion matrix:\n%s' % confusion_matrix(y_test.argmax(axis = 1),prob.argmax(axis = 1)))
print('')
print('Classification Report:')
print(classification_report(y_test.argmax(axis = 1),prob.argmax(axis = 1)))
print('')

Accuracy Rate: 0.635688

Confusion matrix:
[[34  1  1  0  0  0  2]
 [ 0 33  3  1  1  1  0]
 [ 0  7  1 16 12  2  1]
 [ 1  3  0 23  9  1  2]
 [ 1  2  0 23 11  1  0]
 [ 2  0  0  0  0 36  0]
 [ 3  0  1  0  1  0 33]]

Classification Report:
              precision    recall  f1-score   support

           0       0.83      0.89      0.86        38
           1       0.72      0.85      0.78        39
           2       0.17      0.03      0.04        39
           3       0.37      0.59      0.45        39
           4       0.32      0.29      0.31        38
           5       0.88      0.95      0.91        38
           6       0.87      0.87      0.87        38

    accuracy                           0.64       269
   macro avg       0.59      0.64      0.60       269
weighted avg       0.59      0.64      0.60       269




## part 2

Use the same datasets as in the Advanced NN lab and further train NN with different architectures

## Task 1

Try the facial recognition task from NN lab 2 with different model specifications and report model accuracy:
1. Try different dropouts - 0.2, 0.3, 0.4
2. Try different convolution windows: (3$\times$3, 10$\times$10)

In [ ]:
# load the data

from sklearn.datasets import fetch_lfw_people 
from keras.datasets import mnist

lfw_people = fetch_lfw_people(min_faces_per_person = 70, resize = 0.4) 
  
# the images arrays to find the shapes (for plotting) 
n_samples, h, w = lfw_people.images.shape 
  
# Instead of providing 2D data, X has data already in the form  of a vector that 
# is required in this approach. 
X = lfw_people.data 
n_features = X.shape[1] 

y = lfw_people.target 
target_names = lfw_people.target_names 
n_classes = target_names.shape[0] 

In [ ]:
# lets split and reform the dataset in the format compatible for CNN
# basically we are restructuring the data into grid shapes which are apt for convolutions and further steps

X_train, X_test, y_train, y_test = train_test_split( 
    X, y, test_size = 0.2)

X_train_pp = (X_train.reshape(X_train.shape[0], 50, 37, 1).astype('float32')) / 255 # normalize
X_test_pp = (X_test.reshape(X_test.shape[0], 50, 37, 1).astype('float32')) / 255

y_train_pp = np_utils.to_categorical(y_train)
y_test_pp = np_utils.to_categorical(y_test)

num_classes = y_train_pp.shape[1]

In [ ]:
print(X_train_pp.shape, X_test_pp.shape)

(1030, 50, 37, 1) (258, 50, 37, 1)


In [ ]:
Drops=[0.2,0.3,0.4]
Convolution_windows=[(3,3),(10,10)]

In [ ]:
results=[]
for a in Drops:
    for b in Convolution_windows:
        np.random.seed(52)
        model = Sequential()
        model.add(Conv2D(32, b, input_shape=(50, 37, 1), activation='relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Conv2D(64, b, input_shape=(50, 37, 1), activation='relu'))
        model.add(Dropout(a))
        model.add(Flatten())
        model.add(Dense(64, activation='relu'))
        model.add(Dense(num_classes, activation='softmax'))
        # Compile, fit, and generate scores and predicted probabilities.
        model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
        model.fit(X_train_pp, y_train_pp, validation_data=(X_test_pp, y_test_pp), epochs=250, batch_size=200, verbose=2)
        scores = model.evaluate(X_test_pp, y_test_pp, verbose=0)
        prob = model.predict(X_test_pp)
        Results.append((a,b,accuracy_score(y_test,np.argmax(prob,axis=1))))

Epoch 1/250
6/6 - 4s - loss: 1.9017 - accuracy: 0.3456 - val_loss: 1.7675 - val_accuracy: 0.3876 - 4s/epoch - 737ms/step
Epoch 2/250
6/6 - 2s - loss: 1.7174 - accuracy: 0.4175 - val_loss: 1.7561 - val_accuracy: 0.3876 - 2s/epoch - 292ms/step
Epoch 3/250
6/6 - 2s - loss: 1.6918 - accuracy: 0.4175 - val_loss: 1.7505 - val_accuracy: 0.3876 - 2s/epoch - 393ms/step
Epoch 4/250
6/6 - 3s - loss: 1.6774 - accuracy: 0.4175 - val_loss: 1.7549 - val_accuracy: 0.3876 - 3s/epoch - 493ms/step
Epoch 5/250
6/6 - 3s - loss: 1.6770 - accuracy: 0.4175 - val_loss: 1.7284 - val_accuracy: 0.3876 - 3s/epoch - 560ms/step
Epoch 6/250
6/6 - 2s - loss: 1.6761 - accuracy: 0.4175 - val_loss: 1.7321 - val_accuracy: 0.3876 - 2s/epoch - 318ms/step
Epoch 7/250
6/6 - 2s - loss: 1.6756 - accuracy: 0.4175 - val_loss: 1.7387 - val_accuracy: 0.3876 - 2s/epoch - 303ms/step
Epoch 8/250
6/6 - 3s - loss: 1.6738 - accuracy: 0.4175 - val_loss: 1.7280 - val_accuracy: 0.3876 - 3s/epoch - 460ms/step
Epoch 9/250
6/6 - 2s - loss: 1.6

In [ ]:
results

[(0.2, (3, 3), 0.7790697674418605),
 (0.2, (10, 10), 0.7209302325581395),
 (0.3, (3, 3), 0.7674418604651163),
 (0.3, (10, 10), 0.7248062015503876),
 (0.4, (3, 3), 0.7984496124031008),
 (0.4, (10, 10), 0.7441860465116279)]